# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118739e+02     1.804043e+00
 * time: 0.08433699607849121
     1     1.011651e+01     8.955194e-01
 * time: 0.9524109363555908
     2    -1.206517e+01     1.033697e+00
 * time: 1.0834040641784668
     3    -3.421290e+01     8.337126e-01
 * time: 1.4769208431243896
     4    -4.785557e+01     6.107444e-01
 * time: 1.810288906097412
     5    -5.696782e+01     2.298453e-01
 * time: 1.9524140357971191
     6    -5.986501e+01     1.313552e-01
 * time: 2.0636019706726074
     7    -6.099849e+01     5.625435e-02
 * time: 2.1749138832092285
     8    -6.141325e+01     6.920408e-02
 * time: 2.2789900302886963
     9    -6.167948e+01     4.211111e-02
 * time: 2.3901150226593018
    10    -6.187303e+01     2.834969e-02
 * time: 2.5013248920440674
    11    -6.202073e+01     2.089017e-02
 * time: 2.6124119758605957
    12    -6.208090e+01     1.881613e-02
 * time: 2.722761869430542
    13    -6.213760e+01     1.476486e-02
 * time: 2.82636499404

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671062
    AtomicLocal         -18.8557664
    AtomicNonlocal      14.8522641
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485365 
    Xc                  -19.3336818

    total               -62.261666461129
